# Poking the DataCite API with Python

This notebook can be used for minting and deleting DataCite DOIs with the DataCite Member API. To execute a code cell, select it and click the play (black triangle) button in the toolbar. You can also use `Ctrl + Enter`.

Depending on the platform that you're running this notebook on, execution of code cells - especially those that make a request of the DataCite API - may take some time. Be patient, and in bulk operations you might like to get up and do some stretches or get a coffee.

## Initial setup

Create a copy of `datacite-api-config.example.json` named `datacite-api-config.json` and modify it to contain your DataCite Member API username and password. For example:
```json
{
    "username" : "myusername",
    "password" : "mypassword"
}
```

By default this notebook runs on the test API.

Then execute the following code cell to perform initial setup. You swill need to execute this code cell once at the beginning of every session.

In [ ]:
import csv
import datetime
import json
import os
import requests
import urllib

if not os.path.isfile('doilog.csv'):
    with open('doilog.csv', 'w') as csvfile:
        logwriter = csv.writer(csvfile)
        logwriter.writerow([ 'timestamp', 'username', 'filename', 'doi', 'action' ])

with open('datacite-api-config.json') as f:
    data = json.load(f)
    username = data["username"]
    password = data["password"]

use_test_api = True

if use_test_api:
    api_endpoint = 'https://api.test.datacite.org/dois'
else:
    api_endpoint = 'https://api.test.datacite.org/dois'

# Singletons

## Mint a single Draft DOI

The following code cell will let you mint a single DOI in a Draft state. Draft DOIs are not findable, and can be safely deleted.

Create a `metadata.json` file that contains the metadata for your dataset following the [DataCite v4.4 metadata schema](https://schema.datacite.org/meta/kernel-4.4/). You can use [`metadata.example.json`](metadata.example.json) as a guide.

Then, execute the following code cell. If a DOI was successfully minted, it will let you know and log the creation to [`doilog.csv`](doilog.csv).

In [ ]:
filename = 'metadata.json'
url = api_endpoint
headers = {
    'Content-Type': 'application/vnd.api+json',
}
data = open(filename)
response = requests.request("POST", url, auth=(username, password), data = data, headers = headers)
if (response.headers['Status'] == '201 Created'):
    doi = json.loads(response.text)['data']['id']
    timestamp = datetime.datetime.now().replace(microsecond=0).astimezone().isoformat()
    # timestamp = response.headers['Date']
    print(doi + ' minted')
    with open('doilog.csv', 'a') as csvfile:
        logwriter = csv.writer(csvfile)
        logwriter.writerow([ timestamp, username, filename, doi , 'created' ])
else:
    print('DOI not minted')

## Delete a single Draft DOI

Use the following code cells to delete a DOI that is in Draft state. By default the first cell will delete the last DOI you minted in this session.

If you want to delete a specific DOI, put it between the quotes in the second code cell *without* the https://doi.org/. For example, `doi = '10.80335/3vrc-qy79'`. Then, execute the second cell before executing the first cell.

In [ ]:
url = api_endpoint + '/' + urllib.parse.quote_plus(doi)
response = requests.request('DELETE', url, auth=(username, password), headers=headers)
if (response.headers['Status'] == '204 No Content'):
    timestamp = datetime.datetime.now().replace(microsecond=0).astimezone().isoformat()
    # timestamp = response.headers['Date']
    print(doi + ' deleted')
    with open('doilog.csv', 'a') as csvfile:
        logwriter = csv.writer(csvfile)
        logwriter.writerow([ timestamp, username, '', doi , 'deleted' ])
else:
    print(doi + ' not deleted')
    print(response.headers)

In [ ]:
doi = ''

## Register, publish, or hide a single Draft DOI

The following code cell will attempt to register the DOI in the doi variable, which is automatically filled by the single minting process.

Once in Registered or Findable state, a DOI can't be set back to Draft state. This also means that once in Registered or Findable state, a DOI *cannot be delete*. This is serious, mum.

In [ ]:
action = 'hide' # Set this to register, publish, or hide

payload = '{\"data\":{\"attributes\":{\"event\":\"' + action + '\"}}}'
headers = {
    'Content-Type': 'application/vnd.api+json',
}
url = api_endpoint + '/' + urllib.parse.quote_plus(doi)
response = requests.request('PUT', url, auth=(username, password), data = payload, headers=headers)
if (response.headers['Status'] == '200 OK'):
    timestamp = datetime.datetime.now().replace(microsecond=0).astimezone().isoformat()
    # timestamp = response.headers['Date']
    print(doi + ' ' + action + ' successful')
    with open('doilog.csv', 'a') as csvfile:
        logwriter = csv.writer(csvfile)
        logwriter.writerow([ timestamp, username, '', doi , action ])
else:
    print(doi + ' not registered')
    print(response.headers)

In [ ]:
doi = '10.80335/1337'

## Troubleshooting

If you are trying to work out why your DOI was not minted or deleted, execute the following code cell.

In [ ]:
print(response.headers)
print('\n')
print(response.text)
json.loads(response.text)